In [ ]:
# ==================================================
# 🔁 Full Reproducibility + Warning Suppression Setup
# ==================================================
import os
import random
import logging
import warnings

# === ENVIRONMENT VARIABLES (SET BEFORE TF IMPORT) ===
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)           # Hash seed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'           # Suppress TensorFlow logs
os.environ['TF_DETERMINISTIC_OPS'] = '0'           # Allow non-deterministic ops to avoid UnimplementedError
os.environ['CUDA_VISIBLE_DEVICES'] = '0'           # Set GPU ID (or "" to force CPU)

# === PYTHON SEED SETTINGS ===
random.seed(SEED)

# === SUPPRESS WARNINGS & LOGGING ===
logging.getLogger('absl').setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# === IMPORT LIBRARIES AFTER SEED SETTINGS ===
import numpy as np
import tensorflow as tf
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from tqdm import tqdm

# === NUMPY & TENSORFLOW SEEDS ===
np.random.seed(SEED)
tf.random.set_seed(SEED)

# === SINGLE-THREADING FOR FULL REPRODUCIBILITY ===
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

# ✅ CHECK GPU AVAILABILITY
print("✅ GPU Available:", tf.config.list_physical_devices('GPU'))

# ✅ DATASET PATH
DATASET_PATH = r'G:\498R\BanglaSER'

# ✅ EMOTION LABELS FROM FILENAME
# Format: Mode-StatementType-Emotion-Intensity-Statement-Repetition-Actor.wav
EMOTION_MAPPING = {
    '01': 'happy',
    '02': 'sad',
    '03': 'angry',
    '04': 'surprise',
    '05': 'neutral'
}

In [ ]:
# Create dataset information from file structure
def create_dataset_info(root_path):
    data = []
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file.endswith('.wav'):
                # Parse the filename to get emotion
                # Format: Mode-StatementType-Emotion-Intensity-Statement-Repetition-Actor.wav
                parts = file.split('-')
                if len(parts) == 7:
                    emotion_code = parts[2]
                    emotion = EMOTION_MAPPING.get(emotion_code, 'unknown')
                    file_path = os.path.join(root, file)
                    data.append({
                        'file_path': file_path,
                        'emotion': emotion,
                        'emotion_code': emotion_code
                    })
    
    return pd.DataFrame(data)

# Create and display dataset info
df = create_dataset_info(DATASET_PATH)
print(f"Total samples: {len(df)}")
print(df['emotion'].value_counts())

In [ ]:
# ==================================================
# SER model comparison on one dataset (YAMNet vs VGGish)
# Your duplicate-check logic applied:
#   - Global de-dup on RAW AUDIO (after augmentation) BEFORE split
#   - Post-split duplicate detection/cleaning + GRAPHS (your block)
# ==================================================
import os, random, warnings, logging, hashlib
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger('absl').setLevel(logging.ERROR)

# ---------------------------
# Config (EDIT THESE)
# ---------------------------
SEED = 42
DATASET_PATH = r'G:\498R\BanglaSER'       # folder with .wav
SAVE_PATH    = r'G:\498R\exp_outputs'     # where to store npz, figs, results
FEATURE_TYPE = "yamnet"                   # "yamnet" or "vggish"
TARGET_PER_CLASS = 1000                   # augmentation target per class
BATCH_SIZE = 32
DURATION_SEC = 4.0
TARGET_SR = 16000
FIXED_FRAMES = 8                          # ~8 frames for ~4 s clips
RUN_DUP_CHECK_PLOTS = True

os.makedirs(SAVE_PATH, exist_ok=True)

# Reproducibility
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

# ---------------------------
# Dataset discovery + labels
# ---------------------------
# Format: Mode-StatementType-Emotion-Intensity-Statement-Repetition-Actor.wav
EMOTION_MAPPING = {'01':'happy','02':'sad','03':'angry','04':'surprise','05':'neutral'}

def create_dataset_info(root_path):
    rows = []
    for root, _, files in os.walk(root_path):
        for f in files:
            if f.lower().endswith('.wav'):
                parts = f.split('-')
                if len(parts) == 7:
                    code = parts[2]
                    emotion = EMOTION_MAPPING.get(code, 'unknown')
                    rows.append({'file_path': os.path.join(root, f),
                                 'emotion': emotion,
                                 'emotion_code': code})
    return pd.DataFrame(rows)

df = create_dataset_info(DATASET_PATH)
assert len(df) > 0, "No .wav files found."
emotions = sorted(df['emotion'].unique())
label_mapping = {lab:i for i,lab in enumerate(emotions)}
reverse_mapping = {i:lab for lab,i in label_mapping.items()}
num_labels = len(emotions)
print("Label map:", label_mapping)

# ---------------------------
# Audio IO + augmentation
# ---------------------------
TARGET_LEN = int(TARGET_SR * DURATION_SEC)

def load_audio_fixed(path, sr=TARGET_SR, duration=DURATION_SEC):
    y, sr = librosa.load(path, sr=sr, duration=duration, mono=True)
    if len(y) < TARGET_LEN: y = np.pad(y, (0, TARGET_LEN - len(y)))
    else:                   y = y[:TARGET_LEN]
    return y.astype(np.float32, copy=False), sr

def ensure_length(y, L=TARGET_LEN):
    if len(y) < L: y = np.pad(y, (0, L-len(y)))
    elif len(y) > L: y = y[:L]
    return y.astype(np.float32, copy=False)

def aug_noise(y, std=0.005): return (y + np.random.normal(0.0, std, size=y.shape)).astype(np.float32)
def aug_shift(y, max_ratio=0.1):
    m = int(len(y)*max_ratio); s = np.random.randint(-m, m+1)
    if s==0: return y
    if s>0:  return np.concatenate([np.zeros(s, y.dtype), y[:-s]]).astype(np.float32)
    s=-s;    return np.concatenate([y[s:], np.zeros(s, y.dtype)]).astype(np.float32)
def aug_pitch(y, sr=TARGET_SR, rng=(-2,2)):
    steps = np.random.uniform(rng[0], rng[1])
    z = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=steps)
    return ensure_length(z, TARGET_LEN)
def aug_stretch(y, rng=(0.8,1.25)):
    r = np.random.uniform(rng[0], rng[1])
    z = librosa.effects.time_stretch(y=y, rate=r)
    return ensure_length(z, TARGET_LEN)

AUG_FUNCS = ['noise','shift','pitch','stretch']
def apply_aug(y, m):
    return aug_noise(y) if m=='noise' else aug_shift(y) if m=='shift' else aug_pitch(y) if m=='pitch' else aug_stretch(y)

# ---------------------------
# YAMNet/VGGish feature extractors
# ---------------------------
YAMNET_HANDLE = "https://tfhub.dev/google/yamnet/1"
VGGISH_HANDLE = "https://tfhub.dev/google/vggish/1"

print("Loading TF-Hub models…")
yamnet_model = hub.load(YAMNET_HANDLE)
vggish_model = hub.load(VGGISH_HANDLE)

YAMNET_DIM, VGGISH_DIM = 1024, 128

def pad_trunc_frames(X, T=FIXED_FRAMES, D=None):
    # X: (n, d)
    if X is None or X.size == 0:
        d = (X.shape[1] if (X is not None and X.ndim==2 and X.size>0) else (D if D is not None else YAMNET_DIM))
        return np.zeros((T, d), dtype=np.float32)
    n, d = X.shape
    if n >= T: return X[:T]
    out = np.zeros((T,d), dtype=X.dtype)
    out[:n] = X
    return out

def yamnet_embed_frames(y, sr=TARGET_SR):
    if sr != 16000:
        y = librosa.resample(y, orig_sr=sr, target_sr=16000)
    w = tf.convert_to_tensor(y, dtype=tf.float32)
    _, e, _ = yamnet_model(w)          # (n,1024)
    e = e.numpy().astype(np.float32)
    return pad_trunc_frames(e, T=FIXED_FRAMES, D=YAMNET_DIM)

def vggish_embed_frames(y, sr=TARGET_SR):
    if sr != 16000:
        y = librosa.resample(y, orig_sr=sr, target_sr=16000)
    w = tf.convert_to_tensor(y, dtype=tf.float32)
    e = vggish_model(w).numpy().astype(np.float32)  # (n,128)
    return pad_trunc_frames(e, T=FIXED_FRAMES, D=VGGISH_DIM)

def extract_fn_for(feature_type):
    return yamnet_embed_frames if feature_type.lower()=="yamnet" else vggish_embed_frames

# ==================================================
# >>> Global augmentation, then GLOBAL de-dup on RAW AUDIO (your hashing approach) <<<
# ==================================================
def hash_sample_audio(arr, decimals=6):
    """Robust hash for 1D float audio arrays."""
    a = np.asarray(arr, dtype=np.float32)
    if a.ndim == 0:
        a = a.reshape(1)
    a = np.round(a, decimals=decimals)
    a = np.ascontiguousarray(a)
    return hashlib.sha1(a.tobytes()).hexdigest()

def build_augmented_audio_dataset(df, target_per_class=1000):
    """Return raw audio clips AFTER augmentation + global de-dup (audio-level), with labels."""
    raw_audio_list = []   # list of 1D float32 arrays (len = TARGET_LEN)
    labels_list    = []   # int labels
    by_class = defaultdict(list)

    # originals
    print("\nLoading base audio …")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        y, sr = load_audio_fixed(row['file_path'], sr=TARGET_SR, duration=DURATION_SEC)
        lab = label_mapping[row['emotion']]
        raw_audio_list.append(y.astype(np.float32, copy=False))
        labels_list.append(lab)
        by_class[lab].append(y.astype(np.float32, copy=False))

    # augment to balance
    print("Augmenting to balance …")
    for lab, auds in by_class.items():
        need = max(0, target_per_class - len(auds))
        i = 0
        while need > 0:
            base = auds[i % len(auds)]
            m = AUG_FUNCS[np.random.randint(0, len(AUG_FUNCS))]
            ya = apply_aug(base, m)
            ya = ensure_length(ya, TARGET_LEN).astype(np.float32, copy=False)
            raw_audio_list.append(ya)
            labels_list.append(lab)
            i += 1
            need -= 1

    # Keep as list for hashing; labels as ndarray
    labels = np.array(labels_list, dtype=np.int32)

    print("Global audio de-dup (before split) …")
    hashes = [hash_sample_audio(x) for x in raw_audio_list]

    seen, keep_idx = set(), []
    for i, h in enumerate(hashes):
        if h not in seen:
            seen.add(h)
            keep_idx.append(i)
    keep_idx = np.array(keep_idx, dtype=int)

    raw_audio_clean = [raw_audio_list[i] for i in keep_idx]
    labels_clean    = labels[keep_idx]

    print(f"Kept {len(raw_audio_clean)} / {len(raw_audio_list)} after global audio de-dup.")

    return raw_audio_clean, labels_clean

# ==================================================
# Split once (shared by both models)
# ==================================================
SPLITS_FILE = os.path.join(SAVE_PATH, "splits_indices_audiohash.npz")

def get_or_make_splits_shared(labels_clean):
    if os.path.exists(SPLITS_FILE):
        s = np.load(SPLITS_FILE)
        return s['train_idx'], s['val_idx'], s['test_idx']
    idx = np.arange(len(labels_clean))
    tr, tmp, y_tr, y_tmp = train_test_split(idx, labels_clean, test_size=0.2, random_state=SEED, stratify=labels_clean)
    va, te, _, _ = train_test_split(tmp, y_tmp, test_size=0.5, random_state=SEED, stratify=y_tmp)
    np.savez_compressed(SPLITS_FILE, train_idx=tr, val_idx=va, test_idx=te)
    return tr, va, te

# ==================================================
# Build embeddings for the chosen feature type
# ==================================================
def audio_to_embeddings(audio_list, feature_type="yamnet"):
    fn = extract_fn_for(feature_type)
    X = []
    for y in tqdm(audio_list, desc=f"Embedding {feature_type}"):
        X.append(fn(y, TARGET_SR))  # (T,F)
    X = np.array(X, dtype=np.float32)
    return X

# ==================================================
# Scaling + datasets
# ==================================================
def scale_and_make_datasets(X_train, X_val, X_test, y_train, y_val, y_test, batch=BATCH_SIZE):
    T, F = X_train.shape[1], X_train.shape[2]
    scaler = StandardScaler()
    scaler.fit(X_train.reshape(-1, F))

    def tfm(X):
        N = X.shape[0]
        return scaler.transform(X.reshape(-1, F)).reshape(N, T, F).astype(np.float32)

    Xtr, Xva, Xte = tfm(X_train), tfm(X_val), tfm(X_test)
    y_train = y_train.astype(np.int32); y_val = y_val.astype(np.int32); y_test = y_test.astype(np.int32)

    train_ds = tf.data.Dataset.from_tensor_slices((Xtr, y_train)).shuffle(len(Xtr), seed=SEED).batch(batch).prefetch(tf.data.AUTOTUNE)
    val_ds   = tf.data.Dataset.from_tensor_slices((Xva, y_val)).batch(batch).prefetch(tf.data.AUTOTUNE)
    test_ds  = tf.data.Dataset.from_tensor_slices((Xte, y_test)).batch(batch).prefetch(tf.data.AUTOTUNE)
    return Xtr, Xva, Xte, train_ds, val_ds, test_ds, scaler

# ==================================================
# Model + training + eval
# ==================================================
def build_bilstm_classifier(T, F, C):
    inp = layers.Input(shape=(T, F))
    x = layers.Masking()(inp)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(C, activation='softmax')(x)
    model = models.Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def callbacks_for(name):
    return [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(SAVE_PATH, f'best_{name}.h5'),
                                           monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
    ]

def train_and_eval(feature_type, X_train, X_val, X_test, y_train, y_val, y_test, train_ds, val_ds, test_ds):
    T, F = X_train.shape[1], X_train.shape[2]
    name = f"{feature_type.upper()}_BiLSTM_T{T}_F{F}"
    model = build_bilstm_classifier(T, F, num_labels)

    print(f"\nTraining {name} …")
    hist = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=callbacks_for(name), verbose=1)
    model.load_weights(os.path.join(SAVE_PATH, f'best_{name}.h5'))

    # Eval
    eval_out = model.evaluate(test_ds, return_dict=True, verbose=1)
    test_acc = float(eval_out.get('accuracy', np.nan))
    y_prob = model.predict(test_ds, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    y_true = np.concatenate([b.numpy() for _, b in test_ds], axis=0)

    macro_f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Test acc: {test_acc:.4f} | Macro-F1: {macro_f1:.4f}")
    print("\nClassification Report:\n", classification_report(
        y_true, y_pred, target_names=[reverse_mapping[i] for i in range(num_labels)]
    ))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[reverse_mapping[i] for i in range(num_labels)],
                yticklabels=[reverse_mapping[i] for i in range(num_labels)])
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title(f'Confusion Matrix - {name}')
    plt.tight_layout(); plt.savefig(os.path.join(SAVE_PATH, f'{name}_cm.png')); plt.close()

    # Curves
    h = hist.history
    plt.figure(); plt.plot(h['accuracy']); plt.plot(h['val_accuracy']); plt.title(f'{name} Accuracy'); plt.xlabel('epoch'); plt.ylabel('acc'); plt.legend(['train','val']); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, f'{name}_acc.png')); plt.close()
    plt.figure(); plt.plot(h['loss']); plt.plot(h['val_loss']); plt.title(f'{name} Loss'); plt.xlabel('epoch'); plt.ylabel('loss'); plt.legend(['train','val']); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, f'{name}_loss.png')); plt.close()

    # Save row
    res_path = os.path.join(SAVE_PATH, "results.csv")
    row = pd.DataFrame([{"feature_type": feature_type, "T": T, "F": F, "test_accuracy": test_acc, "macro_f1": macro_f1}])
    if os.path.exists(res_path): row.to_csv(res_path, mode='a', header=False, index=False)
    else:                        row.to_csv(res_path, index=False)
    print(f"Saved results to {res_path}")
    return test_acc, macro_f1

# ==================================================
# >>> Build raw audio dataset (augmented) and de-dup globally <<<
# ==================================================
raw_audio_clean, labels_clean = build_augmented_audio_dataset(df, target_per_class=TARGET_PER_CLASS)

# ==================================================
# >>> Get or create shared splits on the CLEANED audio dataset <<<
# ==================================================
train_idx, val_idx, test_idx = get_or_make_splits_shared(labels_clean)

# ==================================================
# >>> Convert cleaned audio to embeddings for chosen model, then slice splits <<<
# ==================================================
X_all = audio_to_embeddings([raw_audio_clean[i] for i in range(len(raw_audio_clean))],
                            feature_type=FEATURE_TYPE)
y_all = labels_clean.copy()

X_train, y_train = X_all[train_idx], y_all[train_idx]
X_val,   y_val   = X_all[val_idx],   y_all[val_idx]
X_test,  y_test  = X_all[test_idx],  y_all[test_idx]

# ==================================================
# >>> Scale + make datasets (scaler fit on TRAIN only)
# ==================================================
X_train, X_val, X_test, train_ds, val_ds, test_ds, scaler = scale_and_make_datasets(
    X_train, X_val, X_test, y_train, y_val, y_test, batch=BATCH_SIZE
)

# ==================================================
# >>> YOUR POST-SPLIT DUPLICATE DETECTION / CLEANING + GRAPHS (unchanged) <<<
# ==================================================
# snapshot "before" for graphs
X_train_bef, y_train_bef = X_train.copy(), y_train.copy()
X_val_bef,   y_val_bef   = X_val.copy(),   y_val.copy()
X_test_bef,  y_test_bef  = X_test.copy(),  y_test.copy()

def hash_sample(arr, decimals=6):
    a = np.ascontiguousarray(np.round(arr, decimals))
    return hashlib.sha1(a.view(np.uint8)).hexdigest()

def hashes_for_split(X):
    return np.array([hash_sample(x) for x in X])

def find_within_split_dups(hashes):
    buckets = defaultdict(list)
    for i, h in enumerate(hashes):
        buckets[h].append(i)
    return {h: idxs for h, idxs in buckets.items() if len(idxs) > 1}

def count_within_extra(dups_dict):
    return sum(len(idxs) - 1 for idxs in dups_dict.values())

def keep_first_indices(hashes):
    seen, keep = set(), []
    for i, h in enumerate(hashes):
        if h not in seen:
            seen.add(h)
            keep.append(i)
    return np.array(keep, dtype=int)

def apply_mask(X, y, keep_idx):
    mask = np.zeros(len(X), dtype=bool)
    mask[keep_idx] = True
    return X[mask], y[mask], mask

# compute hashes BEFORE cleaning
train_h_bef = hashes_for_split(X_train_bef)
val_h_bef   = hashes_for_split(X_val_bef)
test_h_bef  = hashes_for_split(X_test_bef)

dups_train_bef = find_within_split_dups(train_h_bef)
dups_val_bef   = find_within_split_dups(val_h_bef)
dups_test_bef  = find_within_split_dups(test_h_bef)

within_train_bef = count_within_extra(dups_train_bef)
within_val_bef   = count_within_extra(dups_val_bef)
within_test_bef  = count_within_extra(dups_test_bef)

# cross-split overlaps BEFORE cleaning
train_set_bef = set(train_h_bef)
val_set_bef   = set(val_h_bef)
test_set_bef  = set(test_h_bef)

val_in_train_bef  = np.where(np.isin(val_h_bef,  list(train_set_bef)))[0]
test_in_train_bef = np.where(np.isin(test_h_bef, list(train_set_bef)))[0]
train_in_val_bef  = np.where(np.isin(train_h_bef, list(val_set_bef)))[0]
train_in_test_bef = np.where(np.isin(train_h_bef, list(test_set_bef)))[0]
val_in_test_bef   = np.where(np.isin(val_h_bef,  list(test_set_bef)))[0]
test_in_val_bef   = np.where(np.isin(test_h_bef, list(val_set_bef)))[0]

print("\n=== DUPLICATE SUMMARY (before cleaning) ===")
print(f"[TRAIN] within groups: {len(dups_train_bef)} | extras: {within_train_bef}")
print(f"[VAL]   within groups: {len(dups_val_bef)}   | extras: {within_val_bef}")
print(f"[TEST]  within groups: {len(dups_test_bef)}  | extras: {within_test_bef}")
print(f"[VAL]   duplicates in TRAIN: {len(val_in_train_bef)}")
print(f"[TEST]  duplicates in TRAIN: {len(test_in_train_bef)}")
print(f"[TRAIN] duplicates in VAL:   {len(train_in_val_bef)}")
print(f"[TRAIN] duplicates in TEST:  {len(train_in_test_bef)}")
print(f"[VAL]   duplicates in TEST:  {len(val_in_test_bef)}")
print(f"[TEST]  duplicates in VAL:   {len(test_in_val_bef)}")

# Remove duplicates (toggle)
REMOVE_WITHIN_SPLIT = True
REMOVE_VALTEST_IF_IN_TRAIN = True
changed = False

# 1) within-split dedupe
if REMOVE_WITHIN_SPLIT:
    keep_tr = keep_first_indices(train_h_bef)
    keep_va = keep_first_indices(val_h_bef)
    keep_te = keep_first_indices(test_h_bef)

    if len(keep_tr) < len(X_train) or len(keep_va) < len(X_val) or len(keep_te) < len(X_test):
        X_train, y_train, _ = apply_mask(X_train, y_train, keep_tr)
        X_val,   y_val,   _ = apply_mask(X_val,   y_val,   keep_va)
        X_test,  y_test,  _ = apply_mask(X_test,  y_test,  keep_te)
        changed = True

# re-hash after within-split
train_hashes = hashes_for_split(X_train)
val_hashes   = hashes_for_split(X_val)
test_hashes  = hashes_for_split(X_test)
train_set    = set(train_hashes)

# 2) remove any VAL/TEST samples that appear in TRAIN
if REMOVE_VALTEST_IF_IN_TRAIN:
    val_keep_idx  = np.where(~np.isin(val_hashes,  list(train_set)))[0]
    test_keep_idx = np.where(~np.isin(test_hashes, list(train_set)))[0]
    if len(val_keep_idx) < len(X_val) or len(test_keep_idx) < len(X_test):
        X_val,  y_val,  _ = apply_mask(X_val,  y_val,  val_keep_idx)
        X_test, y_test, _ = apply_mask(X_test, y_test, test_keep_idx)
        changed = True

print("\n" + ("Duplicates removed." if changed else "No duplicates removed."))
print(f"Sizes — Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

In [ ]:
# ==================================================
# SER model comparison on one dataset (YAMNet vs VGGish)
# - Offline augmentation on RAW AUDIO for the whole dataset
# - Global audio de-dup BEFORE split (hash on waveform)
# - Shared stratified splits
# - Post-split duplicate detection/cleaning + graphs
# - BiLSTM classifier on top of embeddings
# ==================================================
import os, random, warnings, logging, hashlib
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger('absl').setLevel(logging.ERROR)

# ---------------------------
# Config (EDIT THESE)
# ---------------------------
SEED = 42
DATASET_PATH = r'G:\498R\BanglaSER'       # folder with .wav
SAVE_PATH    = r'G:\498R\exp_outputs'     # where to store npz, figs, results
FEATURE_TYPE = "vggish"                   # "yamnet" or "vggish"
TARGET_PER_CLASS = 1000                   # augmentation target per class
BATCH_SIZE = 32
DURATION_SEC = 4.0
TARGET_SR = 16000
# Optional: different frame counts per model (not required, but tidy)
FIXED_FRAMES = 5 if FEATURE_TYPE.lower()=="vggish" else 8
RUN_DUP_CHECK_PLOTS = True

os.makedirs(SAVE_PATH, exist_ok=True)

# Reproducibility
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

# ---------------------------
# Dataset discovery + labels
# ---------------------------
# Expected filename format:
# Mode-StatementType-Emotion-Intensity-Statement-Repetition-Actor.wav
EMOTION_MAPPING = {'01':'happy','02':'sad','03':'angry','04':'surprise','05':'neutral'}

def create_dataset_info(root_path):
    rows = []
    for root, _, files in os.walk(root_path):
        for f in files:
            if f.lower().endswith('.wav'):
                parts = f.split('-')
                if len(parts) == 7:
                    code = parts[2]
                    emotion = EMOTION_MAPPING.get(code, 'unknown')
                    rows.append({'file_path': os.path.join(root, f),
                                 'emotion': emotion,
                                 'emotion_code': code})
    return pd.DataFrame(rows)

df = create_dataset_info(DATASET_PATH)
assert len(df) > 0, "No .wav files found."
emotions = sorted(df['emotion'].unique())
label_mapping = {lab:i for i,lab in enumerate(emotions)}
reverse_mapping = {i:lab for lab,i in label_mapping.items()}
num_labels = len(emotions)
print("Label map:", label_mapping)

# ---------------------------
# Audio IO + augmentation
# ---------------------------
TARGET_LEN = int(TARGET_SR * DURATION_SEC)

def load_audio_fixed(path, sr=TARGET_SR, duration=DURATION_SEC):
    y, sr = librosa.load(path, sr=sr, duration=duration, mono=True)
    if len(y) < TARGET_LEN: y = np.pad(y, (0, TARGET_LEN - len(y)))
    else:                   y = y[:TARGET_LEN]
    return y.astype(np.float32, copy=False), sr

def ensure_length(y, L=TARGET_LEN):
    if len(y) < L: y = np.pad(y, (0, L-len(y)))
    elif len(y) > L: y = y[:L]
    return y.astype(np.float32, copy=False)

def aug_noise(y, std=0.005): return (y + np.random.normal(0.0, std, size=y.shape)).astype(np.float32)
def aug_shift(y, max_ratio=0.1):
    m = int(len(y)*max_ratio); s = np.random.randint(-m, m+1)
    if s==0: return y
    if s>0:  return np.concatenate([np.zeros(s, y.dtype), y[:-s]]).astype(np.float32)
    s=-s;    return np.concatenate([y[s:], np.zeros(s, y.dtype)]).astype(np.float32)
def aug_pitch(y, sr=TARGET_SR, rng=(-2,2)):
    steps = np.random.uniform(rng[0], rng[1])
    z = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=steps)
    return ensure_length(z, TARGET_LEN)
def aug_stretch(y, rng=(0.8,1.25)):
    r = np.random.uniform(rng[0], rng[1])
    z = librosa.effects.time_stretch(y=y, rate=r)
    return ensure_length(z, TARGET_LEN)

AUG_FUNCS = ['noise','shift','pitch','stretch']
def apply_aug(y, m):
    return aug_noise(y) if m=='noise' else aug_shift(y) if m=='shift' else aug_pitch(y) if m=='pitch' else aug_stretch(y)

# ---------------------------
# YAMNet/VGGish feature extractors
# ---------------------------
YAMNET_HANDLE = "https://tfhub.dev/google/yamnet/1"
VGGISH_HANDLE = "https://tfhub.dev/google/vggish/1"

print("Loading TF-Hub models…")
yamnet_model = hub.load(YAMNET_HANDLE)
vggish_model = hub.load(VGGISH_HANDLE)

YAMNET_DIM, VGGISH_DIM = 1024, 128

def pad_trunc_frames(X, T=FIXED_FRAMES, D=None):
    # X: (n, d)
    if X is None or X.size == 0:
        d = (X.shape[1] if (X is not None and X.ndim==2 and X.size>0) else (D if D is not None else YAMNET_DIM))
        return np.zeros((T, d), dtype=np.float32)
    n, d = X.shape
    if n >= T: return X[:T]
    out = np.zeros((T,d), dtype=X.dtype)
    out[:n] = X
    return out

def yamnet_embed_frames(y, sr=TARGET_SR):
    if sr != 16000:
        y = librosa.resample(y, orig_sr=sr, target_sr=16000)
    w = tf.convert_to_tensor(y, dtype=tf.float32)
    scores, e, _ = yamnet_model(w)          # e: (n,1024)
    e = e.numpy().astype(np.float32)
    return pad_trunc_frames(e, T=FIXED_FRAMES, D=YAMNET_DIM)

def vggish_embed_frames(y, sr=TARGET_SR):
    if sr != 16000:
        y = librosa.resample(y, orig_sr=sr, target_sr=16000)
    w = tf.convert_to_tensor(y, dtype=tf.float32)
    e = vggish_model(w).numpy().astype(np.float32)  # (n,128)
    return pad_trunc_frames(e, T=FIXED_FRAMES, D=VGGISH_DIM)

def extract_fn_for(feature_type):
    return yamnet_embed_frames if feature_type.lower()=="yamnet" else vggish_embed_frames

# ==================================================
# Global augmentation, then GLOBAL de-dup on RAW AUDIO (hashing)
# ==================================================
def hash_sample_audio(arr, decimals=6):
    """Robust hash for 1D float audio arrays."""
    a = np.asarray(arr, dtype=np.float32)
    if a.ndim == 0:
        a = a.reshape(1)
    a = np.round(a, decimals=decimals)
    a = np.ascontiguousarray(a)
    return hashlib.sha1(a.tobytes()).hexdigest()

def build_augmented_audio_dataset(df, target_per_class=1000):
    """Return raw audio clips AFTER augmentation + global de-dup (audio-level), with labels."""
    raw_audio_list = []   # list of 1D float32 arrays (len = TARGET_LEN)
    labels_list    = []   # int labels
    by_class = defaultdict(list)

    # originals
    print("\nLoading base audio …")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        y, sr = load_audio_fixed(row['file_path'], sr=TARGET_SR, duration=DURATION_SEC)
        lab = label_mapping[row['emotion']]
        raw_audio_list.append(y.astype(np.float32, copy=False))
        labels_list.append(lab)
        by_class[lab].append(y.astype(np.float32, copy=False))

    # augment to balance
    print("Augmenting to balance …")
    for lab, auds in by_class.items():
        need = max(0, target_per_class - len(auds))
        i = 0
        while need > 0:
            base = auds[i % len(auds)]
            m = AUG_FUNCS[np.random.randint(0, len(AUG_FUNCS))]
            ya = apply_aug(base, m)
            ya = ensure_length(ya, TARGET_LEN).astype(np.float32, copy=False)
            raw_audio_list.append(ya)
            labels_list.append(lab)
            i += 1
            need -= 1

    labels = np.array(labels_list, dtype=np.int32)

    print("Global audio de-dup (before split) …")
    hashes = [hash_sample_audio(x) for x in raw_audio_list]
    seen, keep_idx = set(), []
    for i, h in enumerate(hashes):
        if h not in seen:
            seen.add(h)
            keep_idx.append(i)
    keep_idx = np.array(keep_idx, dtype=int)

    raw_audio_clean = [raw_audio_list[i] for i in keep_idx]
    labels_clean    = labels[keep_idx]

    print(f"Kept {len(raw_audio_clean)} / {len(raw_audio_list)} after global audio de-dup.")
    return raw_audio_clean, labels_clean

# ==================================================
# Split once (shared by both models)
# ==================================================
SPLITS_FILE = os.path.join(SAVE_PATH, "splits_indices_audiohash.npz")

def get_or_make_splits_shared(labels_clean):
    if os.path.exists(SPLITS_FILE):
        s = np.load(SPLITS_FILE)
        return s['train_idx'], s['val_idx'], s['test_idx']
    idx = np.arange(len(labels_clean))
    tr, tmp, y_tr, y_tmp = train_test_split(idx, labels_clean, test_size=0.2, random_state=SEED, stratify=labels_clean)
    va, te, _, _ = train_test_split(tmp, y_tmp, test_size=0.5, random_state=SEED, stratify=y_tmp)
    np.savez_compressed(SPLITS_FILE, train_idx=tr, val_idx=va, test_idx=te)
    return tr, va, te

# ==================================================
# Build embeddings for the chosen feature type
# ==================================================
def audio_to_embeddings(audio_list, feature_type="yamnet"):
    fn = extract_fn_for(feature_type)
    X = []
    for y in tqdm(audio_list, desc=f"Embedding {feature_type}"):
        X.append(fn(y, TARGET_SR))  # (T,F)
    X = np.array(X, dtype=np.float32)
    return X

# ==================================================
# Scaling + datasets
# ==================================================
def scale_and_make_datasets(X_train, X_val, X_test, y_train, y_val, y_test, batch=BATCH_SIZE):
    T, F = X_train.shape[1], X_train.shape[2]
    scaler = StandardScaler()
    scaler.fit(X_train.reshape(-1, F))

    def tfm(X):
        N = X.shape[0]
        return scaler.transform(X.reshape(-1, F)).reshape(N, T, F).astype(np.float32)

    Xtr, Xva, Xte = tfm(X_train), tfm(X_val), tfm(X_test)
    y_train = y_train.astype(np.int32); y_val = y_val.astype(np.int32); y_test = y_test.astype(np.int32)

    train_ds = tf.data.Dataset.from_tensor_slices((Xtr, y_train)).shuffle(len(Xtr), seed=SEED).batch(batch).prefetch(tf.data.AUTOTUNE)
    val_ds   = tf.data.Dataset.from_tensor_slices((Xva, y_val)).batch(batch).prefetch(tf.data.AUTOTUNE)
    test_ds  = tf.data.Dataset.from_tensor_slices((Xte, y_test)).batch(batch).prefetch(tf.data.AUTOTUNE)
    return Xtr, Xva, Xte, train_ds, val_ds, test_ds, scaler

# ==================================================
# Model + training + eval
# ==================================================
def build_bilstm_classifier(T, F, C):
    inp = layers.Input(shape=(T, F))
    x = layers.Masking()(inp)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(C, activation='softmax')(x)
    model = models.Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def callbacks_for(name):
    return [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(SAVE_PATH, f'best_{name}.h5'),
                                           monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
    ]

def train_and_eval(feature_type, X_train, X_val, X_test, y_train, y_val, y_test, train_ds, val_ds, test_ds):
    T, F = X_train.shape[1], X_train.shape[2]
    name = f"{feature_type.upper()}_BiLSTM_T{T}_F{F}"
    model = build_bilstm_classifier(T, F, num_labels)

    print(f"\nTraining {name} …")
    hist = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=callbacks_for(name), verbose=1)
    model.load_weights(os.path.join(SAVE_PATH, f'best_{name}.h5'))

    # Eval
    eval_out = model.evaluate(test_ds, return_dict=True, verbose=1)
    test_acc = float(eval_out.get('accuracy', np.nan))
    y_prob = model.predict(test_ds, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    y_true = np.concatenate([b.numpy() for _, b in test_ds], axis=0)

    macro_f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Test acc: {test_acc:.4f} | Macro-F1: {macro_f1:.4f}")
    print("\nClassification Report:\n", classification_report(
        y_true, y_pred, target_names=[reverse_mapping[i] for i in range(num_labels)], digits=4 
    ))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[reverse_mapping[i] for i in range(num_labels)],
                yticklabels=[reverse_mapping[i] for i in range(num_labels)])
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title(f'Confusion Matrix - {name}')
    plt.tight_layout(); plt.savefig(os.path.join(SAVE_PATH, f'{name}_cm.png')); plt.close()

    # Curves
    h = hist.history
    plt.figure(); plt.plot(h['accuracy']); plt.plot(h['val_accuracy']); plt.title(f'{name} Accuracy'); plt.xlabel('epoch'); plt.ylabel('acc'); plt.legend(['train','val']); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, f'{name}_acc.png')); plt.close()
    plt.figure(); plt.plot(h['loss']); plt.plot(h['val_loss']); plt.title(f'{name} Loss'); plt.xlabel('epoch'); plt.ylabel('loss'); plt.legend(['train','val']); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, f'{name}_loss.png')); plt.close()

    # Save row
    res_path = os.path.join(SAVE_PATH, "results.csv")
    row = pd.DataFrame([{"feature_type": feature_type, "T": T, "F": F, "test_accuracy": test_acc, "macro_f1": macro_f1}])
    if os.path.exists(res_path): row.to_csv(res_path, mode='a', header=False, index=False)
    else:                        row.to_csv(res_path, index=False)
    print(f"Saved results to {res_path}")
    return test_acc, macro_f1

# ==================================================
# >>> Build raw audio dataset (augmented) and de-dup globally <<<
# ==================================================
raw_audio_clean, labels_clean = build_augmented_audio_dataset(df, target_per_class=TARGET_PER_CLASS)

# ==================================================
# >>> Get or create shared splits on the CLEANED audio dataset <<<
# ==================================================
train_idx, val_idx, test_idx = get_or_make_splits_shared(labels_clean)

# ==================================================
# >>> Convert cleaned audio to embeddings for chosen model, then slice splits <<<
# ==================================================
X_all = audio_to_embeddings([raw_audio_clean[i] for i in range(len(raw_audio_clean))],
                            feature_type=FEATURE_TYPE)
y_all = labels_clean.copy()

X_train, y_train = X_all[train_idx], y_all[train_idx]
X_val,   y_val   = X_all[val_idx],   y_all[val_idx]
X_test,  y_test  = X_all[test_idx],  y_all[test_idx]

# ==================================================
# >>> Scale + make datasets (scaler fit on TRAIN only)
# ==================================================
X_train, X_val, X_test, train_ds, val_ds, test_ds, scaler = scale_and_make_datasets(
    X_train, X_val, X_test, y_train, y_val, y_test, batch=BATCH_SIZE
)

# ==================================================
# >>> POST-SPLIT DUPLICATE DETECTION / CLEANING + GRAPHS <<<
# ==================================================
# snapshot "before" for graphs
X_train_bef, y_train_bef = X_train.copy(), y_train.copy()
X_val_bef,   y_val_bef   = X_val.copy(),   y_val.copy()
X_test_bef,  y_test_bef  = X_test.copy(),  y_test.copy()

def hash_sample(arr, decimals=6):
    a = np.ascontiguousarray(np.round(arr, decimals))
    return hashlib.sha1(a.view(np.uint8)).hexdigest()

def hashes_for_split(X):
    return np.array([hash_sample(x) for x in X])

def find_within_split_dups(hashes):
    buckets = defaultdict(list)
    for i, h in enumerate(hashes):
        buckets[h].append(i)
    return {h: idxs for h, idxs in buckets.items() if len(idxs) > 1}

def count_within_extra(dups_dict):
    return sum(len(idxs) - 1 for idxs in dups_dict.values())

def keep_first_indices(hashes):
    seen, keep = set(), []
    for i, h in enumerate(hashes):
        if h not in seen:
            seen.add(h)
            keep.append(i)
    return np.array(keep, dtype=int)

def apply_mask(X, y, keep_idx):
    mask = np.zeros(len(X), dtype=bool)
    mask[keep_idx] = True
    return X[mask], y[mask], mask

# compute hashes BEFORE cleaning
train_h_bef = hashes_for_split(X_train_bef)
val_h_bef   = hashes_for_split(X_val_bef)
test_h_bef  = hashes_for_split(X_test_bef)

dups_train_bef = find_within_split_dups(train_h_bef)
dups_val_bef   = find_within_split_dups(val_h_bef)
dups_test_bef  = find_within_split_dups(test_h_bef)

within_train_bef = count_within_extra(dups_train_bef)
within_val_bef   = count_within_extra(dups_val_bef)
within_test_bef  = count_within_extra(dups_test_bef)

# cross-split overlaps BEFORE cleaning
train_set_bef = set(train_h_bef)
val_set_bef   = set(val_h_bef)
test_set_bef  = set(test_h_bef)

val_in_train_bef  = np.where(np.isin(val_h_bef,  list(train_set_bef)))[0]
test_in_train_bef = np.where(np.isin(test_h_bef, list(train_set_bef)))[0]
train_in_val_bef  = np.where(np.isin(train_h_bef, list(val_set_bef)))[0]
train_in_test_bef = np.where(np.isin(train_h_bef, list(test_set_bef)))[0]
val_in_test_bef   = np.where(np.isin(val_h_bef,  list(test_set_bef)))[0]
test_in_val_bef   = np.where(np.isin(test_h_bef, list(val_set_bef)))[0]

print("\n=== DUPLICATE SUMMARY (before cleaning) ===")
print(f"[TRAIN] within groups: {len(dups_train_bef)} | extras: {within_train_bef}")
print(f"[VAL]   within groups: {len(dups_val_bef)}   | extras: {within_val_bef}")
print(f"[TEST]  within groups: {len(dups_test_bef)}  | extras: {within_test_bef}")
print(f"[VAL]   duplicates in TRAIN: {len(val_in_train_bef)}")
print(f"[TEST]  duplicates in TRAIN: {len(test_in_train_bef)}")
print(f"[TRAIN] duplicates in VAL:   {len(train_in_val_bef)}")
print(f"[TRAIN] duplicates in TEST:  {len(train_in_test_bef)}")
print(f"[VAL]   duplicates in TEST:  {len(val_in_test_bef)}")
print(f"[TEST]  duplicates in VAL:   {len(test_in_val_bef)}")

# Remove duplicates (toggle)
REMOVE_WITHIN_SPLIT = True
REMOVE_VALTEST_IF_IN_TRAIN = True
changed = False

# 1) within-split dedupe
if REMOVE_WITHIN_SPLIT:
    keep_tr = keep_first_indices(train_h_bef)
    keep_va = keep_first_indices(val_h_bef)
    keep_te = keep_first_indices(test_h_bef)

    if len(keep_tr) < len(X_train) or len(keep_va) < len(X_val) or len(keep_te) < len(X_test):
        X_train, y_train, _ = apply_mask(X_train, y_train, keep_tr)
        X_val,   y_val,   _ = apply_mask(X_val,   y_val,   keep_va)
        X_test,  y_test,  _ = apply_mask(X_test,  y_test,  keep_te)
        changed = True

# re-hash after within-split
train_hashes = hashes_for_split(X_train)
val_hashes   = hashes_for_split(X_val)
test_hashes  = hashes_for_split(X_test)
train_set    = set(train_hashes)

# 2) remove any VAL/TEST samples that appear in TRAIN
if REMOVE_VALTEST_IF_IN_TRAIN:
    val_keep_idx  = np.where(~np.isin(val_hashes,  list(train_set)))[0]
    test_keep_idx = np.where(~np.isin(test_hashes, list(train_set)))[0]
    if len(val_keep_idx) < len(X_val) or len(test_keep_idx) < len(X_test):
        X_val,  y_val,  _ = apply_mask(X_val,  y_val,  val_keep_idx)
        X_test, y_test, _ = apply_mask(X_test, y_test, test_keep_idx)
        changed = True

print("\n" + ("Duplicates removed." if changed else "No duplicates removed."))
print(f"Sizes — Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# --- POST-clean for graphs ---
train_h_aft = hashes_for_split(X_train)
val_h_aft   = hashes_for_split(X_val)
test_h_aft  = hashes_for_split(X_test)

dups_train_aft = find_within_split_dups(train_h_aft)
dups_val_aft   = find_within_split_dups(val_h_aft)
dups_test_aft  = find_within_split_dups(test_h_aft)

within_train_aft = count_within_extra(dups_train_aft)
within_val_aft   = count_within_extra(dups_val_aft)
within_test_aft  = count_within_extra(dups_test_aft)

# GRAPHS
if RUN_DUP_CHECK_PLOTS:
    # 1) Counts before vs after
    sizes_before = [len(X_train_bef), len(X_val_bef), len(X_test_bef)]
    sizes_after  = [len(X_train),     len(X_val),     len(X_test)]
    splits = ['Train', 'Val', 'Test']

    plt.figure(figsize=(8,5))
    x = np.arange(len(splits)); w = 0.38
    plt.bar(x - w/2, sizes_before, width=w, label='Before')
    plt.bar(x + w/2, sizes_after,  width=w, label='After')
    plt.xticks(x, splits); plt.ylabel('Num samples')
    plt.title('Split sizes: before vs after duplicate cleaning')
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, 'dup_sizes_before_after.png')); plt.close()

    # 2) Within-split duplicates (extras) before vs after
    within_bef = [within_train_bef, within_val_bef, within_test_bef]
    within_aft = [within_train_aft, within_val_aft, within_test_aft]

    plt.figure(figsize=(8,5))
    plt.bar(x - w/2, within_bef, width=w, label='Before')
    plt.bar(x + w/2, within_aft, width=w, label='After')
    plt.xticks(x, splits); plt.ylabel('Num duplicate extras')
    plt.title('Within-split duplicate extras: before vs after')
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, 'dup_within_extras_before_after.png')); plt.close()

    # 3) Cross-split overlap heatmap (BEFORE cleaning)
    cross_overlap_before = np.array([
        [within_train_bef, len(val_in_train_bef), len(test_in_train_bef)],
        [len(train_in_val_bef), within_val_bef, len(val_in_test_bef)],
        [len(train_in_test_bef), len(test_in_val_bef), within_test_bef]
    ], dtype=int)

    plt.figure(figsize=(7,6))
    sns.heatmap(cross_overlap_before, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Train','Val','Test'],
                yticklabels=['Train','Val','Test'])
    plt.title('Duplicate overlap (BEFORE cleaning)\n(diagonal = within extras)')
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, 'dup_overlap_heatmap_before.png')); plt.close()

    # 4) Class distribution per split — before and after (stacked bars)
    def plot_class_dist(y_train, y_val, y_test, title, fname):
        classes = [reverse_mapping[i] for i in range(len(reverse_mapping))]
        counts_tr = np.array([np.sum(y_train == i) for i in range(len(reverse_mapping))])
        counts_va = np.array([np.sum(y_val   == i) for i in range(len(reverse_mapping))])
        counts_te = np.array([np.sum(y_test  == i) for i in range(len(reverse_mapping))])

        plt.figure(figsize=(10,6))
        bottoms = np.zeros(3)
        for i, cls in enumerate(classes):
            vals = np.array([counts_tr[i], counts_va[i], counts_te[i]])
            plt.bar(['Train','Val','Test'], vals, bottom=bottoms, label=cls)
            bottoms += vals
        plt.ylabel('Num samples'); plt.title(title)
        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.tight_layout(); plt.savefig(os.path.join(SAVE_PATH, fname)); plt.close()

    plot_class_dist(y_train_bef, y_val_bef, y_test_bef,
                    'Class distribution per split (BEFORE cleaning)',
                    'class_dist_before.png')

    plot_class_dist(y_train, y_val, y_test,
                    'Class distribution per split (AFTER cleaning)',
                    'class_dist_after.png')

    print("📈 Saved figures:",
          "dup_sizes_before_after.png,",
          "dup_within_extras_before_after.png,",
          "dup_overlap_heatmap_before.png,",
          "class_dist_before.png,",
          "class_dist_after.png")

# ==================================================
# >>> Rebuild tf.data datasets after cleaning <<<
# ==================================================
X_train_cln, X_val_cln, X_test_cln = X_train, X_val, X_test  # already cleaned above
train_ds = tf.data.Dataset.from_tensor_slices((X_train_cln, y_train)).shuffle(len(X_train_cln), seed=SEED).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val_cln,   y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test_cln,  y_test)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ==================================================
# >>> Train & evaluate
# ==================================================
acc, f1 = train_and_eval(FEATURE_TYPE, X_train_cln, X_val_cln, X_test_cln, y_train, y_val, y_test,
                         train_ds, val_ds, test_ds)

print(f"\nDone: {FEATURE_TYPE.upper()} | acc={acc:.4f} | macro_f1={f1:.4f}")
print("Tip: change FEATURE_TYPE to 'yamnet' (or 'vggish') and run again. Splits are shared.")

In [ ]:
**Yamnet**

In [ ]:
# ==================================================
# SER model comparison on one dataset (YAMNet vs VGGish)
# - Offline augmentation on RAW AUDIO for the whole dataset
# - Global audio de-dup BEFORE split (hash on waveform)
# - Shared stratified splits
# - Post-split duplicate detection/cleaning + graphs
# - BiLSTM classifier on top of embeddings
# ==================================================
import os, random, warnings, logging, hashlib
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger('absl').setLevel(logging.ERROR)

# ---------------------------
# Config (EDIT THESE)
# ---------------------------
SEED = 42
DATASET_PATH = r'G:\498R\BanglaSER'       # folder with .wav
SAVE_PATH    = r'G:\498R\exp_outputs'     # where to store npz, figs, results
FEATURE_TYPE = "yamnet"                   # "yamnet" or "vggish"
TARGET_PER_CLASS = 1000                   # augmentation target per class
BATCH_SIZE = 32
DURATION_SEC = 4.0
TARGET_SR = 16000
# Optional: different frame counts per model (not required, but tidy)
FIXED_FRAMES = 5 if FEATURE_TYPE.lower()=="vggish" else 8
RUN_DUP_CHECK_PLOTS = True

os.makedirs(SAVE_PATH, exist_ok=True)

# Reproducibility
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

# ---------------------------
# Dataset discovery + labels
# ---------------------------
# Expected filename format:
# Mode-StatementType-Emotion-Intensity-Statement-Repetition-Actor.wav
EMOTION_MAPPING = {'01':'happy','02':'sad','03':'angry','04':'surprise','05':'neutral'}

def create_dataset_info(root_path):
    rows = []
    for root, _, files in os.walk(root_path):
        for f in files:
            if f.lower().endswith('.wav'):
                parts = f.split('-')
                if len(parts) == 7:
                    code = parts[2]
                    emotion = EMOTION_MAPPING.get(code, 'unknown')
                    rows.append({'file_path': os.path.join(root, f),
                                 'emotion': emotion,
                                 'emotion_code': code})
    return pd.DataFrame(rows)

df = create_dataset_info(DATASET_PATH)
assert len(df) > 0, "No .wav files found."
emotions = sorted(df['emotion'].unique())
label_mapping = {lab:i for i,lab in enumerate(emotions)}
reverse_mapping = {i:lab for lab,i in label_mapping.items()}
num_labels = len(emotions)
print("Label map:", label_mapping)

# ---------------------------
# Audio IO + augmentation
# ---------------------------
TARGET_LEN = int(TARGET_SR * DURATION_SEC)

def load_audio_fixed(path, sr=TARGET_SR, duration=DURATION_SEC):
    y, sr = librosa.load(path, sr=sr, duration=duration, mono=True)
    if len(y) < TARGET_LEN: y = np.pad(y, (0, TARGET_LEN - len(y)))
    else:                   y = y[:TARGET_LEN]
    return y.astype(np.float32, copy=False), sr

def ensure_length(y, L=TARGET_LEN):
    if len(y) < L: y = np.pad(y, (0, L-len(y)))
    elif len(y) > L: y = y[:L]
    return y.astype(np.float32, copy=False)

def aug_noise(y, std=0.005): return (y + np.random.normal(0.0, std, size=y.shape)).astype(np.float32)
def aug_shift(y, max_ratio=0.1):
    m = int(len(y)*max_ratio); s = np.random.randint(-m, m+1)
    if s==0: return y
    if s>0:  return np.concatenate([np.zeros(s, y.dtype), y[:-s]]).astype(np.float32)
    s=-s;    return np.concatenate([y[s:], np.zeros(s, y.dtype)]).astype(np.float32)
def aug_pitch(y, sr=TARGET_SR, rng=(-2,2)):
    steps = np.random.uniform(rng[0], rng[1])
    z = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=steps)
    return ensure_length(z, TARGET_LEN)
def aug_stretch(y, rng=(0.8,1.25)):
    r = np.random.uniform(rng[0], rng[1])
    z = librosa.effects.time_stretch(y=y, rate=r)
    return ensure_length(z, TARGET_LEN)

AUG_FUNCS = ['noise','shift','pitch','stretch']
def apply_aug(y, m):
    return aug_noise(y) if m=='noise' else aug_shift(y) if m=='shift' else aug_pitch(y) if m=='pitch' else aug_stretch(y)

# ---------------------------
# YAMNet/VGGish feature extractors
# ---------------------------
YAMNET_HANDLE = "https://tfhub.dev/google/yamnet/1"
VGGISH_HANDLE = "https://tfhub.dev/google/vggish/1"

print("Loading TF-Hub models…")
yamnet_model = hub.load(YAMNET_HANDLE)
vggish_model = hub.load(VGGISH_HANDLE)

YAMNET_DIM, VGGISH_DIM = 1024, 128

def pad_trunc_frames(X, T=FIXED_FRAMES, D=None):
    # X: (n, d)
    if X is None or X.size == 0:
        d = (X.shape[1] if (X is not None and X.ndim==2 and X.size>0) else (D if D is not None else YAMNET_DIM))
        return np.zeros((T, d), dtype=np.float32)
    n, d = X.shape
    if n >= T: return X[:T]
    out = np.zeros((T,d), dtype=X.dtype)
    out[:n] = X
    return out

def yamnet_embed_frames(y, sr=TARGET_SR):
    if sr != 16000:
        y = librosa.resample(y, orig_sr=sr, target_sr=16000)
    w = tf.convert_to_tensor(y, dtype=tf.float32)
    scores, e, _ = yamnet_model(w)          # e: (n,1024)
    e = e.numpy().astype(np.float32)
    return pad_trunc_frames(e, T=FIXED_FRAMES, D=YAMNET_DIM)

def vggish_embed_frames(y, sr=TARGET_SR):
    if sr != 16000:
        y = librosa.resample(y, orig_sr=sr, target_sr=16000)
    w = tf.convert_to_tensor(y, dtype=tf.float32)
    e = vggish_model(w).numpy().astype(np.float32)  # (n,128)
    return pad_trunc_frames(e, T=FIXED_FRAMES, D=VGGISH_DIM)

def extract_fn_for(feature_type):
    return yamnet_embed_frames if feature_type.lower()=="yamnet" else vggish_embed_frames

# ==================================================
# Global augmentation, then GLOBAL de-dup on RAW AUDIO (hashing)
# ==================================================
def hash_sample_audio(arr, decimals=6):
    """Robust hash for 1D float audio arrays."""
    a = np.asarray(arr, dtype=np.float32)
    if a.ndim == 0:
        a = a.reshape(1)
    a = np.round(a, decimals=decimals)
    a = np.ascontiguousarray(a)
    return hashlib.sha1(a.tobytes()).hexdigest()

def build_augmented_audio_dataset(df, target_per_class=1000):
    """Return raw audio clips AFTER augmentation + global de-dup (audio-level), with labels."""
    raw_audio_list = []   # list of 1D float32 arrays (len = TARGET_LEN)
    labels_list    = []   # int labels
    by_class = defaultdict(list)

    # originals
    print("\nLoading base audio …")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        y, sr = load_audio_fixed(row['file_path'], sr=TARGET_SR, duration=DURATION_SEC)
        lab = label_mapping[row['emotion']]
        raw_audio_list.append(y.astype(np.float32, copy=False))
        labels_list.append(lab)
        by_class[lab].append(y.astype(np.float32, copy=False))

    # augment to balance
    print("Augmenting to balance …")
    for lab, auds in by_class.items():
        need = max(0, target_per_class - len(auds))
        i = 0
        while need > 0:
            base = auds[i % len(auds)]
            m = AUG_FUNCS[np.random.randint(0, len(AUG_FUNCS))]
            ya = apply_aug(base, m)
            ya = ensure_length(ya, TARGET_LEN).astype(np.float32, copy=False)
            raw_audio_list.append(ya)
            labels_list.append(lab)
            i += 1
            need -= 1

    labels = np.array(labels_list, dtype=np.int32)

    print("Global audio de-dup (before split) …")
    hashes = [hash_sample_audio(x) for x in raw_audio_list]
    seen, keep_idx = set(), []
    for i, h in enumerate(hashes):
        if h not in seen:
            seen.add(h)
            keep_idx.append(i)
    keep_idx = np.array(keep_idx, dtype=int)

    raw_audio_clean = [raw_audio_list[i] for i in keep_idx]
    labels_clean    = labels[keep_idx]

    print(f"Kept {len(raw_audio_clean)} / {len(raw_audio_list)} after global audio de-dup.")
    return raw_audio_clean, labels_clean

# ==================================================
# Split once (shared by both models)
# ==================================================
SPLITS_FILE = os.path.join(SAVE_PATH, "splits_indices_audiohash.npz")

def get_or_make_splits_shared(labels_clean):
    if os.path.exists(SPLITS_FILE):
        s = np.load(SPLITS_FILE)
        return s['train_idx'], s['val_idx'], s['test_idx']
    idx = np.arange(len(labels_clean))
    tr, tmp, y_tr, y_tmp = train_test_split(idx, labels_clean, test_size=0.2, random_state=SEED, stratify=labels_clean)
    va, te, _, _ = train_test_split(tmp, y_tmp, test_size=0.5, random_state=SEED, stratify=y_tmp)
    np.savez_compressed(SPLITS_FILE, train_idx=tr, val_idx=va, test_idx=te)
    return tr, va, te

# ==================================================
# Build embeddings for the chosen feature type
# ==================================================
def audio_to_embeddings(audio_list, feature_type="yamnet"):
    fn = extract_fn_for(feature_type)
    X = []
    for y in tqdm(audio_list, desc=f"Embedding {feature_type}"):
        X.append(fn(y, TARGET_SR))  # (T,F)
    X = np.array(X, dtype=np.float32)
    return X

# ==================================================
# Scaling + datasets
# ==================================================
def scale_and_make_datasets(X_train, X_val, X_test, y_train, y_val, y_test, batch=BATCH_SIZE):
    T, F = X_train.shape[1], X_train.shape[2]
    scaler = StandardScaler()
    scaler.fit(X_train.reshape(-1, F))

    def tfm(X):
        N = X.shape[0]
        return scaler.transform(X.reshape(-1, F)).reshape(N, T, F).astype(np.float32)

    Xtr, Xva, Xte = tfm(X_train), tfm(X_val), tfm(X_test)
    y_train = y_train.astype(np.int32); y_val = y_val.astype(np.int32); y_test = y_test.astype(np.int32)

    train_ds = tf.data.Dataset.from_tensor_slices((Xtr, y_train)).shuffle(len(Xtr), seed=SEED).batch(batch).prefetch(tf.data.AUTOTUNE)
    val_ds   = tf.data.Dataset.from_tensor_slices((Xva, y_val)).batch(batch).prefetch(tf.data.AUTOTUNE)
    test_ds  = tf.data.Dataset.from_tensor_slices((Xte, y_test)).batch(batch).prefetch(tf.data.AUTOTUNE)
    return Xtr, Xva, Xte, train_ds, val_ds, test_ds, scaler

# ==================================================
# Model + training + eval
# ==================================================
def build_bilstm_classifier(T, F, C):
    inp = layers.Input(shape=(T, F))
    x = layers.Masking()(inp)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(C, activation='softmax')(x)
    model = models.Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def callbacks_for(name):
    return [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(SAVE_PATH, f'best_{name}.h5'),
                                           monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
    ]

def train_and_eval(feature_type, X_train, X_val, X_test, y_train, y_val, y_test, train_ds, val_ds, test_ds):
    T, F = X_train.shape[1], X_train.shape[2]
    name = f"{feature_type.upper()}_BiLSTM_T{T}_F{F}"
    model = build_bilstm_classifier(T, F, num_labels)

    print(f"\nTraining {name} …")
    hist = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=callbacks_for(name), verbose=1)
    model.load_weights(os.path.join(SAVE_PATH, f'best_{name}.h5'))

    # Eval
    eval_out = model.evaluate(test_ds, return_dict=True, verbose=1)
    test_acc = float(eval_out.get('accuracy', np.nan))
    y_prob = model.predict(test_ds, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    y_true = np.concatenate([b.numpy() for _, b in test_ds], axis=0)

    macro_f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Test acc: {test_acc:.4f} | Macro-F1: {macro_f1:.4f}")
    print("\nClassification Report:\n", classification_report(
        y_true, y_pred, target_names=[reverse_mapping[i] for i in range(num_labels)], digits=4   # <-- 4 decimal places
    ))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[reverse_mapping[i] for i in range(num_labels)],
                yticklabels=[reverse_mapping[i] for i in range(num_labels)])
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title(f'Confusion Matrix - {name}')
    plt.tight_layout(); plt.savefig(os.path.join(SAVE_PATH, f'{name}_cm.png')); plt.close()

    # Curves
    h = hist.history
    plt.figure(); plt.plot(h['accuracy']); plt.plot(h['val_accuracy']); plt.title(f'{name} Accuracy'); plt.xlabel('epoch'); plt.ylabel('acc'); plt.legend(['train','val']); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, f'{name}_acc.png')); plt.close()
    plt.figure(); plt.plot(h['loss']); plt.plot(h['val_loss']); plt.title(f'{name} Loss'); plt.xlabel('epoch'); plt.ylabel('loss'); plt.legend(['train','val']); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, f'{name}_loss.png')); plt.close()

    # Save row
    res_path = os.path.join(SAVE_PATH, "results.csv")
    row = pd.DataFrame([{"feature_type": feature_type, "T": T, "F": F, "test_accuracy": test_acc, "macro_f1": macro_f1}])
    if os.path.exists(res_path): row.to_csv(res_path, mode='a', header=False, index=False)
    else:                        row.to_csv(res_path, index=False)
    print(f"Saved results to {res_path}")
    return test_acc, macro_f1

# ==================================================
# >>> Build raw audio dataset (augmented) and de-dup globally <<<
# ==================================================
raw_audio_clean, labels_clean = build_augmented_audio_dataset(df, target_per_class=TARGET_PER_CLASS)

# ==================================================
# >>> Get or create shared splits on the CLEANED audio dataset <<<
# ==================================================
train_idx, val_idx, test_idx = get_or_make_splits_shared(labels_clean)

# ==================================================
# >>> Convert cleaned audio to embeddings for chosen model, then slice splits <<<
# ==================================================
X_all = audio_to_embeddings([raw_audio_clean[i] for i in range(len(raw_audio_clean))],
                            feature_type=FEATURE_TYPE)
y_all = labels_clean.copy()

X_train, y_train = X_all[train_idx], y_all[train_idx]
X_val,   y_val   = X_all[val_idx],   y_all[val_idx]
X_test,  y_test  = X_all[test_idx],  y_all[test_idx]

# ==================================================
# >>> Scale + make datasets (scaler fit on TRAIN only)
# ==================================================
X_train, X_val, X_test, train_ds, val_ds, test_ds, scaler = scale_and_make_datasets(
    X_train, X_val, X_test, y_train, y_val, y_test, batch=BATCH_SIZE
)

# ==================================================
# >>> POST-SPLIT DUPLICATE DETECTION / CLEANING + GRAPHS <<<
# ==================================================
# snapshot "before" for graphs
X_train_bef, y_train_bef = X_train.copy(), y_train.copy()
X_val_bef,   y_val_bef   = X_val.copy(),   y_val.copy()
X_test_bef,  y_test_bef  = X_test.copy(),  y_test.copy()

def hash_sample(arr, decimals=6):
    a = np.ascontiguousarray(np.round(arr, decimals))
    return hashlib.sha1(a.view(np.uint8)).hexdigest()

def hashes_for_split(X):
    return np.array([hash_sample(x) for x in X])

def find_within_split_dups(hashes):
    buckets = defaultdict(list)
    for i, h in enumerate(hashes):
        buckets[h].append(i)
    return {h: idxs for h, idxs in buckets.items() if len(idxs) > 1}

def count_within_extra(dups_dict):
    return sum(len(idxs) - 1 for idxs in dups_dict.values())

def keep_first_indices(hashes):
    seen, keep = set(), []
    for i, h in enumerate(hashes):
        if h not in seen:
            seen.add(h)
            keep.append(i)
    return np.array(keep, dtype=int)

def apply_mask(X, y, keep_idx):
    mask = np.zeros(len(X), dtype=bool)
    mask[keep_idx] = True
    return X[mask], y[mask], mask

# compute hashes BEFORE cleaning
train_h_bef = hashes_for_split(X_train_bef)
val_h_bef   = hashes_for_split(X_val_bef)
test_h_bef  = hashes_for_split(X_test_bef)

dups_train_bef = find_within_split_dups(train_h_bef)
dups_val_bef   = find_within_split_dups(val_h_bef)
dups_test_bef  = find_within_split_dups(test_h_bef)

within_train_bef = count_within_extra(dups_train_bef)
within_val_bef   = count_within_extra(dups_val_bef)
within_test_bef  = count_within_extra(dups_test_bef)

# cross-split overlaps BEFORE cleaning
train_set_bef = set(train_h_bef)
val_set_bef   = set(val_h_bef)
test_set_bef  = set(test_h_bef)

val_in_train_bef  = np.where(np.isin(val_h_bef,  list(train_set_bef)))[0]
test_in_train_bef = np.where(np.isin(test_h_bef, list(train_set_bef)))[0]
train_in_val_bef  = np.where(np.isin(train_h_bef, list(val_set_bef)))[0]
train_in_test_bef = np.where(np.isin(train_h_bef, list(test_set_bef)))[0]
val_in_test_bef   = np.where(np.isin(val_h_bef,  list(test_set_bef)))[0]
test_in_val_bef   = np.where(np.isin(test_h_bef, list(val_set_bef)))[0]

print("\n=== DUPLICATE SUMMARY (before cleaning) ===")
print(f"[TRAIN] within groups: {len(dups_train_bef)} | extras: {within_train_bef}")
print(f"[VAL]   within groups: {len(dups_val_bef)}   | extras: {within_val_bef}")
print(f"[TEST]  within groups: {len(dups_test_bef)}  | extras: {within_test_bef}")
print(f"[VAL]   duplicates in TRAIN: {len(val_in_train_bef)}")
print(f"[TEST]  duplicates in TRAIN: {len(test_in_train_bef)}")
print(f"[TRAIN] duplicates in VAL:   {len(train_in_val_bef)}")
print(f"[TRAIN] duplicates in TEST:  {len(train_in_test_bef)}")
print(f"[VAL]   duplicates in TEST:  {len(val_in_test_bef)}")
print(f"[TEST]  duplicates in VAL:   {len(test_in_val_bef)}")

# Remove duplicates (toggle)
REMOVE_WITHIN_SPLIT = True
REMOVE_VALTEST_IF_IN_TRAIN = True
changed = False

# 1) within-split dedupe
if REMOVE_WITHIN_SPLIT:
    keep_tr = keep_first_indices(train_h_bef)
    keep_va = keep_first_indices(val_h_bef)
    keep_te = keep_first_indices(test_h_bef)

    if len(keep_tr) < len(X_train) or len(keep_va) < len(X_val) or len(keep_te) < len(X_test):
        X_train, y_train, _ = apply_mask(X_train, y_train, keep_tr)
        X_val,   y_val,   _ = apply_mask(X_val,   y_val,   keep_va)
        X_test,  y_test,  _ = apply_mask(X_test,  y_test,  keep_te)
        changed = True

# re-hash after within-split
train_hashes = hashes_for_split(X_train)
val_hashes   = hashes_for_split(X_val)
test_hashes  = hashes_for_split(X_test)
train_set    = set(train_hashes)

# 2) remove any VAL/TEST samples that appear in TRAIN
if REMOVE_VALTEST_IF_IN_TRAIN:
    val_keep_idx  = np.where(~np.isin(val_hashes,  list(train_set)))[0]
    test_keep_idx = np.where(~np.isin(test_hashes, list(train_set)))[0]
    if len(val_keep_idx) < len(X_val) or len(test_keep_idx) < len(X_test):
        X_val,  y_val,  _ = apply_mask(X_val,  y_val,  val_keep_idx)
        X_test, y_test, _ = apply_mask(X_test, y_test, test_keep_idx)
        changed = True

print("\n" + ("Duplicates removed." if changed else "No duplicates removed."))
print(f"Sizes — Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# --- POST-clean for graphs ---
train_h_aft = hashes_for_split(X_train)
val_h_aft   = hashes_for_split(X_val)
test_h_aft  = hashes_for_split(X_test)

dups_train_aft = find_within_split_dups(train_h_aft)
dups_val_aft   = find_within_split_dups(val_h_aft)
dups_test_aft  = find_within_split_dups(test_h_aft)

within_train_aft = count_within_extra(dups_train_aft)
within_val_aft   = count_within_extra(dups_val_aft)
within_test_aft  = count_within_extra(dups_test_aft)

# GRAPHS
if RUN_DUP_CHECK_PLOTS:
    # 1) Counts before vs after
    sizes_before = [len(X_train_bef), len(X_val_bef), len(X_test_bef)]
    sizes_after  = [len(X_train),     len(X_val),     len(X_test)]
    splits = ['Train', 'Val', 'Test']

    plt.figure(figsize=(8,5))
    x = np.arange(len(splits)); w = 0.38
    plt.bar(x - w/2, sizes_before, width=w, label='Before')
    plt.bar(x + w/2, sizes_after,  width=w, label='After')
    plt.xticks(x, splits); plt.ylabel('Num samples')
    plt.title('Split sizes: before vs after duplicate cleaning')
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, 'dup_sizes_before_after.png')); plt.close()

    # 2) Within-split duplicates (extras) before vs after
    within_bef = [within_train_bef, within_val_bef, within_test_bef]
    within_aft = [within_train_aft, within_val_aft, within_test_aft]

    plt.figure(figsize=(8,5))
    plt.bar(x - w/2, within_bef, width=w, label='Before')
    plt.bar(x + w/2, within_aft, width=w, label='After')
    plt.xticks(x, splits); plt.ylabel('Num duplicate extras')
    plt.title('Within-split duplicate extras: before vs after')
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, 'dup_within_extras_before_after.png')); plt.close()

    # 3) Cross-split overlap heatmap (BEFORE cleaning)
    cross_overlap_before = np.array([
        [within_train_bef, len(val_in_train_bef), len(test_in_train_bef)],
        [len(train_in_val_bef), within_val_bef, len(val_in_test_bef)],
        [len(train_in_test_bef), len(test_in_val_bef), within_test_bef]
    ], dtype=int)

    plt.figure(figsize=(7,6))
    sns.heatmap(cross_overlap_before, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Train','Val','Test'],
                yticklabels=['Train','Val','Test'])
    plt.title('Duplicate overlap (BEFORE cleaning)\n(diagonal = within extras)')
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, 'dup_overlap_heatmap_before.png')); plt.close()

    # 4) Class distribution per split — before and after (stacked bars)
    def plot_class_dist(y_train, y_val, y_test, title, fname):
        classes = [reverse_mapping[i] for i in range(len(reverse_mapping))]
        counts_tr = np.array([np.sum(y_train == i) for i in range(len(reverse_mapping))])
        counts_va = np.array([np.sum(y_val   == i) for i in range(len(reverse_mapping))])
        counts_te = np.array([np.sum(y_test  == i) for i in range(len(reverse_mapping))])

        plt.figure(figsize=(10,6))
        bottoms = np.zeros(3)
        for i, cls in enumerate(classes):
            vals = np.array([counts_tr[i], counts_va[i], counts_te[i]])
            plt.bar(['Train','Val','Test'], vals, bottom=bottoms, label=cls)
            bottoms += vals
        plt.ylabel('Num samples'); plt.title(title)
        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.tight_layout(); plt.savefig(os.path.join(SAVE_PATH, fname)); plt.close()

    plot_class_dist(y_train_bef, y_val_bef, y_test_bef,
                    'Class distribution per split (BEFORE cleaning)',
                    'class_dist_before.png')

    plot_class_dist(y_train, y_val, y_test,
                    'Class distribution per split (AFTER cleaning)',
                    'class_dist_after.png')

    print("📈 Saved figures:",
          "dup_sizes_before_after.png,",
          "dup_within_extras_before_after.png,",
          "dup_overlap_heatmap_before.png,",
          "class_dist_before.png,",
          "class_dist_after.png")

# ==================================================
# >>> Rebuild tf.data datasets after cleaning <<<
# ==================================================
X_train_cln, X_val_cln, X_test_cln = X_train, X_val, X_test  # already cleaned above
train_ds = tf.data.Dataset.from_tensor_slices((X_train_cln, y_train)).shuffle(len(X_train_cln), seed=SEED).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val_cln,   y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test_cln,  y_test)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ==================================================
# >>> Train & evaluate
# ==================================================
acc, f1 = train_and_eval(FEATURE_TYPE, X_train_cln, X_val_cln, X_test_cln, y_train, y_val, y_test,
                         train_ds, val_ds, test_ds)

print(f"\nDone: {FEATURE_TYPE.upper()} | acc={acc:.4f} | macro_f1={f1:.4f}")
print("Tip: change FEATURE_TYPE to 'yamnet' (or 'vggish') and run again. Splits are shared.")